# 5. Binary Search Trees, BST Sort

- Runway Reservation Problem:
    - Airport with a single, busy runway
    - We need a data structure (a set) that will allow us to keep track of airplane landings. Data structure contains times (let the times be a monotonically increasing function for simplicity)
    - When the plane lands, we remove it from the set of pending landings
    - When a landing request comes in, place the time of request in the data structure ONLY IF no other landings are scheduled within k minutes (let k be a variable you'd put into the insert function but for the example consider k = 3)
    - Operations needed:
        - insert()
        - remove()
    - SPECIFICATION: All operations need to be done in O(log(n)) time
- Problems with Different Data Structures for the RRP?
    - Unsorted array:
        - the k minute check would take _O(n)_ which is > _O(log(n))_
    - Sorted array:
        - could do binary search to find where to insert a time --> would take _O(log(n))_ time to find insertion point
        - the k minute check would take _O(1)_ time
        - BUT the insertion would need to shift all elements to the right one over which would take _O(n)_ time
    - Sorted list (linked list):
        - can insert in _O(1)_ time by moving pointers
        - BUT can't do binary search on a list
    - Heaps
        - min/max heap has WEAK invariant in terms of children as opposed to left and right subtrees as is the case with binary search tree
        - if you want to find where to put a new time, t into the data structure (we would need to find an element that is <= k or >= k from t), it would take _O(n)_ time.
- **Binary Search Tree:**
    - Data structure needs:
        - node x
            - each node has a key which can be returned through key(x)
        - pointers
            - parent(x) --> parent of a node
            - left(x) --> left child
            - right(x) --> right child
    - INVARIANT:
        - for all nodes x
            - if y is in the left subtree of x, key(y) <= key(x)
            - if y is in the right subtree of x, key(y) >= key(x)
    - Some operations:
        - find_min() --> keep going to the left until you hit a leaf and return the key of the leaf. 
        - find_max() --> keep going to the right until you hit a leaf and the return the key of the leaf
        - next_larger() --> return the key of the node to the right
    - BSTs are augmentable --> can alter to fit into problem specifications
    - BSTs and RRP:
        - before inserting an element, t, in the BST, we would run a check at the node we are inserting at, parent(t), is within k minutes of t.
            - IF IT IS, we can insert, ELSE, can not insert
        - the insertion with the check operation would take _O(h)_ time where h is the height of the tree
            - WHY h and not log(n)?
                - need balanced trees in order for it be O(log(n))
                - h could very well be n which is a problem
        - NOW, let's say the problem statement changed and we now want this extra feature Rank(t)
            - Rank(t) --> returns # of planes that are scheduled to land at times <= t
            - We augment BST structure so that each keeps track of the number of sub-nodes under it + itself
                - <img src = "screenshots/img_4.png" width = 400>
                - as a result, adding an element would change the numbers associated with each node
            - To do Rank(t) we would follow the procedure detailed in the above picture.
                - It would take _O(h)_ time
            - Rank(t) example: --> Rank(79) in the above
                - @ 49 --> add 1 AND add 2 (from the 46 subtree)
                - @ 79 --> add 1, 79 <= 79 AND subtree-size to the left (subtree 64)

## Recitation Notes:
- Data Structure: 
    - bunch of algorithms that let you store and retrieve informations
    - 2 types of algorithms:
        - queries: ask questions
        - updates: add data 
            - must ensure the RI holds
    - They have representation invariant (RI or rep. invariant)
        - says that data is organized in a certain way and as long as we follow those rules, the data structure functions correctly
        - TIP:
            - when implementing a data structure, implement a method during debugging called check_RI that walks through it and checks if the rep. invariant holds throughout
            - put check_RI calls in every update method when debugging
- BST:
    - queries:
        - MAX
        - MIN
        - NEXT-LARGER
        - SEARCH
    - updates:
        - INSERT
            - Complexity is O(h)
        - DELETE
            - Complexity is O(h)
                - search for key is O(h)
                - at worst, you'll need to find successor and perform a swap. This is also O(h)
    - rep_invariant:
        - for all nodes x
            - if y is in the left subtree of x, key(y) <= key(x)
            - if y is in the right subtree of x, key(y) >= key(x)
    - need 2 things
        - Node class
            - keep track of left_child, right_child, parent, key
        - BST class
            - made of nodes

In [4]:
# Simplest implementation of BST

class BST(object):
    """
Simple binary search tree implementation.
This BST supports insert, find, and delete-min operations.
Each tree contains some (possibly 0) BSTnode objects, representing nodes,
and a pointer to the root.
"""

    def __init__(self):
        self.root = None

    def insert(self, t):
        """Insert key t into this BST, modifying it in-place."""
        new = BSTnode(t)
        if self.root is None:
            self.root = new
        else:
            node = self.root
            while True: # not recursive, in a while loop
                if t < node.key:
                    # Go left
                    if node.left is None:
                        node.left = new
                        new.parent = node
                        break
                    node = node.left
                else:
                    # Go right
                    if node.right is None:
                        node.right = new
                        new.parent = node
                        break
                    node = node.right
        return new

    def find(self, t):
        """Return the node for key t if is in the tree, or None otherwise."""
        node = self.root
        while node is not None:
            if t == node.key:
                return node
            elif t < node.key:
                node = node.left
            else:
                node = node.right
        return None

    def delete_min(self):
        """Delete the minimum key (and return the old node containing it)."""
        if self.root is None:
            return None, None
        else:
            # Walk to leftmost node.
            node = self.root
            while node.left is not None:
                node = node.left
            # Remove that node and promote its right subtree.
            if node.parent is not None:
                node.parent.left = node.right
            else: # The root was smallest.
                self.root = node.right
            if node.right is not None:
                node.right.parent = node.parent
            parent = node.parent
            node.disconnect()
            return node, parent

    def __str__(self):
        if self.root is None: return '<empty tree>'
        def recurse(node):
            if node is None: return [], 0, 0
            label = str(node.key)
            left_lines, left_pos, left_width = recurse(node.left)
            right_lines, right_pos, right_width = recurse(node.right)
            middle = max(right_pos + left_width - left_pos + 1, len(label), 2)
            pos = left_pos + middle // 2
            width = left_pos + middle + right_width - right_pos
            while len(left_lines) < len(right_lines):
                left_lines.append(' ' * left_width)
            while len(right_lines) < len(left_lines):
                right_lines.append(' ' * right_width)
            if (middle - len(label)) % 2 == 1 and node.parent is not None and \
               node is node.parent.left and len(label) < middle:
                label += '.'
            label = label.center(middle, '.')
            if label[0] == '.': label = ' ' + label[1:]
            if label[-1] == '.': label = label[:-1] + ' '
            lines = [' ' * left_pos + label + ' ' * (right_width - right_pos),
                     ' ' * left_pos + '/' + ' ' * (middle-2) +
                     '\\' + ' ' * (right_width - right_pos)] + \
              [left_line + ' ' * (width - left_width - right_width) +
               right_line
               for left_line, right_line in zip(left_lines, right_lines)]
            return lines, pos, width
        return '\n'.join(recurse(self.root) [0])

class BSTnode(object):
    """
Representation of a node in a binary search tree.
Has a left child, right child, and key value.
"""
    def __init__(self, t):
        """Create a new leaf with key t."""
        self.key = t
        self.disconnect()
        
    def disconnect(self):
        self.left = None
        self.right = None
        self.parent = None